<a href="https://colab.research.google.com/github/Santosdevbjj/relatoVendasLucros/blob/main/notebooks/simulacao_dashboard_sem_powerbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# ---
# Simulação de Dashboard Power BI com Python + Plotly Dash
# Autor: Sérgio Santos
# Projeto: Relatório de Vendas e Lucros — Data Analytics
# ---

# ============================================
# 1️⃣ Importação das bibliotecas
# ============================================

import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash

# ============================================
# 2️⃣ Carregar e preparar os dados
# ============================================

df = pd.read_csv("../data/vendas_tratadas.csv")

# Converter data para datetime
df["Data"] = pd.to_datetime(df["Data"])

# Criar colunas de tempo derivadas
df["Ano"] = df["Data"].dt.year
df["Mes"] = df["Data"].dt.month_name()

# KPIs globais
receita_total = df["Receita"].sum()
lucro_total = df["Lucro"].sum()
margem_media = (df["Lucro"] / df["Receita"]).mean() * 100

# ============================================
# 3️⃣ Construção do aplicativo Dash
# ============================================

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("📊 Relatório de Vendas e Lucros", style={"textAlign": "center", "color": "#1f4e79"}),
    html.H3("Simulação de Dashboard Power BI — Python + Plotly Dash", style={"textAlign": "center", "color": "gray"}),

    html.Br(),

    # KPIs
    html.Div([
        html.Div([
            html.H4("Receita Total"),
            html.H2(f"R$ {receita_total:,.2f}", style={"color": "#007ACC"})
        ], className="card"),

        html.Div([
            html.H4("Lucro Total"),
            html.H2(f"R$ {lucro_total:,.2f}", style={"color": "#1DB954"})
        ], className="card"),

        html.Div([
            html.H4("Margem Média"),
            html.H2(f"{margem_media:.2f}%", style={"color": "#FFB000"})
        ], className="card"),
    ], style={"display": "flex", "justifyContent": "space-around", "padding": "10px"}),

    html.Br(),

    # Filtros
    html.Div([
        html.Div([
            html.Label("📅 Selecione o Ano:"),
            dcc.Dropdown(
                id="filtro-ano",
                options=[{"label": ano, "value": ano} for ano in sorted(df["Ano"].unique())],
                value=df["Ano"].max(),
                clearable=False
            )
        ], style={"width": "30%", "display": "inline-block"}),

        html.Div([
            html.Label("🌎 Selecione a Região:"),
            dcc.Dropdown(
                id="filtro-regiao",
                options=[{"label": reg, "value": reg} for reg in sorted(df["Nome_Regiao"].unique())],
                value=None,
                placeholder="Todas as regiões",
                multi=True
            )
        ], style={"width": "60%", "display": "inline-block", "marginLeft": "20px"}),
    ], style={"padding": "20px"}),

    html.Br(),

    # Gráficos
    html.Div([
        dcc.Graph(id="grafico_receita_area", style={"width": "48%", "display": "inline-block"}),
        dcc.Graph(id="grafico_lucro_categoria", style={"width": "48%", "display": "inline-block", "float": "right"}),
    ]),

    html.Br(),

    html.Div([
        dcc.Graph(id="grafico_mapa_regioes")
    ])
])

# ============================================
# 4️⃣ Callbacks interativos
# ============================================

@app.callback(
    [Output("grafico_receita_area", "figure"),
     Output("grafico_lucro_categoria", "figure"),
     Output("grafico_mapa_regioes", "figure")],
    [Input("filtro-ano", "value"),
     Input("filtro-regiao", "value")]
)
def atualizar_graficos(ano, regioes):
    # Filtrar por ano
    df_filtrado = df[df["Ano"] == ano]

    # Filtrar por região (caso selecionada)
    if regioes:
        df_filtrado = df_filtrado[df_filtrado["Nome_Regiao"].isin(regioes)]

    # === Gráfico 1: Receita por mês e região ===
    fig_receita = px.area(
        df_filtrado.groupby(["Mes", "Nome_Regiao"], as_index=False)["Receita"].sum(),
        x="Mes", y="Receita", color="Nome_Regiao",
        title=f"Receita Mensal por Região — {ano}"
    )
    fig_receita.update_layout(template="plotly_white", legend_title="Região")

    # === Gráfico 2: Lucro por categoria ===
    fig_lucro = px.bar(
        df_filtrado.groupby("Categoria", as_index=False)["Lucro"].sum(),
        x="Categoria", y="Lucro", color="Categoria",
        title="Lucro Total por Categoria de Produto"
    )
    fig_lucro.update_layout(template="plotly_white", showlegend=False)

    # === Gráfico 3: Mapa (Simulação de localização) ===
    # Para efeito didático, criaremos coordenadas médias para cada região
    coordenadas = {
        "Sudeste": (-23.55, -46.63),
        "Sul": (-25.43, -49.27),
        "Nordeste": (-12.97, -38.51),
        "Centro-Oeste": (-16.68, -49.25)
    }
    df_coords = df_filtrado.groupby("Nome_Regiao")[["Receita", "Lucro"]].sum().reset_index()
    df_coords["lat"] = df_coords["Nome_Regiao"].map(lambda r: coordenadas[r][0])
    df_coords["lon"] = df_coords["Nome_Regiao"].map(lambda r: coordenadas[r][1])

    fig_mapa = px.scatter_geo(
        df_coords, lat="lat", lon="lon",
        size="Lucro", color="Nome_Regiao",
        hover_name="Nome_Regiao",
        projection="natural earth",
        title="Distribuição de Lucros por Região (Mapa Simulado)"
    )

    return fig_receita, fig_lucro, fig_mapa

# ============================================
# 5️⃣ Executar o aplicativo
# ============================================

app.run_server(mode="inline", debug=False)